In [1]:
import numpy as np
import collections
import os
import matplotlib.pyplot as plt


from constants import DT, XML_DIR, START_ARM_POSE
from constants import PUPPET_GRIPPER_POSITION_CLOSE
from constants import PUPPET_GRIPPER_POSITION_UNNORMALIZE_FN
from constants import PUPPET_GRIPPER_POSITION_NORMALIZE_FN
from constants import PUPPET_GRIPPER_VELOCITY_NORMALIZE_FN

from utils import sample_box_pose, sample_insertion_pose
from dm_control import mujoco
from dm_control.rl import control
from dm_control.suite import base
from dm_control import viewer
import PIL.Image

START_ARM_POSE = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
class BimanualViperXTask(base.Task):
    def __init__(self, random=None):
        super().__init__(random=random)

    def before_step(self, action, physics):
        env_action = action
        super().before_step(env_action, physics)
        return

    def initialize_episode(self, physics):
        """Sets the state of the environment at the start of each episode."""
        super().initialize_episode(physics)

    @staticmethod
    def get_qpos(physics):
        qpos_raw = physics.data.qpos.copy()
        return qpos_raw[:18]

    @staticmethod
    def get_qvel(physics):
        qvel_raw = physics.data.qvel.copy()
        return qvel_raw[:18]

    @staticmethod
    def get_env_state(physics):
        raise NotImplementedError

    def get_observation(self, physics):
        obs = collections.OrderedDict()
        obs['qpos'] = self.get_qpos(physics)
        obs['qvel'] = self.get_qvel(physics)
        obs['env_state'] = self.get_env_state(physics)
        obs['images'] = dict()
        obs['images']['top'] = physics.render(height=480, width=640, camera_id='top')
        obs['images']['angle'] = physics.render(height=480, width=640, camera_id='angle')
        obs['images']['vis'] = physics.render(height=480, width=640, camera_id='fl_dabai')

        return obs

    def get_reward(self, physics):
        # return whether left gripper is holding the box
        raise NotImplementedError

from math import pi


class TransferCubeTask(BimanualViperXTask):
    def __init__(self, phi, random=None):
        super().__init__(random=random)
        self.max_reward = 10
        self.phi = phi

    def initialize_episode(self, physics):
        """Sets the state of the environment at the start of each episode."""
        # TODO Notice: this function does not randomize the env configuration. Instead, set BOX_POSE from outside
        # reset qpos, control and box position
        with physics.reset_context():
            x, y = self.place_cube()
            BOX_POSE = [[x, y, 0.8, 1, 0, 0, 0]]
            physics.named.data.qpos[:18] = START_ARM_POSE
            np.copyto(physics.data.ctrl, START_ARM_POSE)
            assert BOX_POSE[0] is not None
            physics.named.data.qpos[-7:] = BOX_POSE[0]
            # print(f"{BOX_POSE=}")
        super().initialize_episode(physics)
    
    def place_cube(self):
        r0 = np.array([0.233, 0.3])
        R = np.sqrt((0.52 - 0.233) ** 2 + (0.1 - 0.3) ** 2)
        return r0 + np.array([R * np.cos(-self.phi), R * np.sin(-self.phi)])

    @staticmethod
    def get_env_state(physics):
        env_state = physics.data.qpos.copy()[20:]
        return env_state

    def get_reward(self, physics):
        # return whether left gripper is holding the box
        all_contact_pairs = []
        for i_contact in range(physics.data.ncon):
            id_geom_1 = physics.data.contact[i_contact].geom1
            id_geom_2 = physics.data.contact[i_contact].geom2
            name_geom_1 = physics.model.id2name(id_geom_1, 'geom')
            name_geom_2 = physics.model.id2name(id_geom_2, 'geom')
            contact_pair = (name_geom_1, name_geom_2)
            all_contact_pairs.append(contact_pair)

        touch_left_gripper = ("red_box", "fl_7") in all_contact_pairs
        cube_touch_table = ("red_box", "table2") in all_contact_pairs
        robot_touch_table = ("footprint_fixed_joint_lump__box2_Link_collision_1", "table2") in all_contact_pairs

        reward = 0
        if touch_left_gripper: # attempted transfer
            reward = 3
        if robot_touch_table: # attempted transfer
            reward = 4
        if cube_touch_table: # successful transfer
            reward = 10
        return reward


In [11]:
phi = 0.0314159 * (-24 + 0)
task = TransferCubeTask(phi, random=False)
xml_path = os.path.join(XML_DIR, 'aloha_model.xml')
physics = mujoco.Physics.from_xml_path(xml_path)
    
env = control.Environment(physics, task, time_limit=20, control_timestep=DT, n_sub_steps=None, flat_observation=False)
ts = env.reset()


In [12]:
viewer.launch(env)

In [21]:
import time
import h5py
import cv2
num_episodes = 50
episode_len = 850
render_cam_name = 'top'
success = []

for episode_idx in range(num_episodes):
    video = []
    print(f'{episode_idx=}')
    print('Rollout out EE space scripted policy')
        # setup the environment
    phi = 0.0314159 * (-24 + episode_idx)
    task = TransferCubeTask(phi, random=False)
    xml_path = os.path.join(XML_DIR, 'aloha_model.xml')
    physics = mujoco.Physics.from_xml_path(xml_path)
    
    env = control.Environment(physics, task, time_limit=20, control_timestep=DT, n_sub_steps=None, flat_observation=False)
    ts = env.reset()
    episode_replay = [ts]

    pos0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    pos1 = np.array([0, 0,-phi + 0.03, 1.8, 1.3, -0.9, 0, pi / 2 - phi, 0.04, 0.04, 0, 0, 0, 0, 0, 0, 0, 0])
    pos2 = np.array([0, 0,-phi, 1.9, 1.3, -0.9, 0, pi / 2 - phi, 0.017, 0.017, 0, 0, 0, 0, 0, 0, 0, 0])
    pos3 = np.array([0, 0,0, 0, 0, 0, 0, 1.57, 0.017, 0.017, 0, 0, 0, 0, 0, 0, 0, 0])
    pos4 = np.array([0, 0,-1.57, 0, 0, 0, 0, 1.57, 0.019, 0.019, 0, 0, 0, 0, 0, 0, 0, 0])
    pos5 = np.array([-1, -1,-1.57, 0, 0, 0, 0, 1.57, 0.019, 0.019, 0, 0, 0, 0, 0, 0, 0, 0])
    pos6 = np.array([1, -1,-1.57, 0, 0, 0, 0, 1.57, 0.019, 0.019, 0, 0, 0, 0, 0, 0, 0, 0])

    for step in range(50):
        action = pos0 * (1 - step / 50) + pos1 * (step / 50) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])

    for step in range(50):
        action = pos1 * (1 - step / 50) + pos2 * (step / 50) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])

    for step in range(100):
        action = pos2 * (1 - step / 100) + pos3 * (step / 100) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])

    for step in range(50):
        action = pos3 * (1 - step / 50) + pos4 * (step / 50) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])

    for step in range(10):
        action = pos4 * (1 - step / 10) + pos5 * (step / 10) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])
    
    for step in range(100):
        action = pos5
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])
    
    for step in range(10):
        action = pos5 * (1 - step / 10) + pos6 * (step / 10) 
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])
    
    for step in range(300):
        action = pos6
        ts = env.step(action)
        episode_replay.append(ts)
        video.append(ts.observation['images'])

    joint_traj = [ts.observation['qpos'] for ts in episode_replay]
    episode_return = np.sum([ts.reward for ts in episode_replay[1:] if not (ts.reward is None)])
    episode_max_reward = np.max([ts.reward for ts in episode_replay[1:] if not (ts.reward is None)])
    if episode_max_reward == env.task.max_reward:
        success.append(1)
        print(f"{episode_idx=} Successful, {episode_return=}")
    else:
        success.append(0)
        print(f"{episode_idx=} Failed")
    data_dict = {
            '/observations/qpos': [],
            '/observations/qvel': [],
            '/action': [],
    }
    camera_names = ['top', 'angle', 'vis']
    for cam_name in camera_names:
        data_dict[f'/observations/images/{cam_name}'] = []

        # because the replaying, there will be eps_len + 1 actions and eps_len + 2 timesteps
        # truncate here to be consistent
    joint_traj = joint_traj[:-1]
    episode_replay = episode_replay[:-1]
        # len(joint_traj) i.e. actions: max_timesteps
        # len(episode_replay) i.e. time steps: max_timesteps + 1
    max_timesteps = len(joint_traj)
    while joint_traj:
        action = joint_traj.pop(0)
        ts = episode_replay.pop(0)
        data_dict['/observations/qpos'].append(ts.observation['qpos'])
        data_dict['/observations/qvel'].append(ts.observation['qvel'])
        data_dict['/action'].append(action)
        for cam_name in camera_names:
            data_dict[f'/observations/images/{cam_name}'].append(ts.observation['images'][cam_name])
            
    t0 = time.time()
    dataset_dir = '/media/jacob/hdd/aloha_rollouts'
    dataset_path = os.path.join(dataset_dir, f'episode_{episode_idx}')
    with h5py.File(dataset_path + '.hdf5', 'w', rdcc_nbytes=1024 ** 2 * 2) as root:
        root.attrs['sim'] = True
        obs = root.create_group('observations')
        image = obs.create_group('images')
        for cam_name in camera_names:
            _ = image.create_dataset(cam_name, (max_timesteps, 480, 640, 3), dtype='uint8',
                                         chunks=(1, 480, 640, 3), )
            # compression='gzip',compression_opts=2,)
            # compression=32001, compression_opts=(0, 0, 0, 0, 9, 1, 1), shuffle=False)
        qpos = obs.create_dataset('qpos', (max_timesteps, 18))
        qvel = obs.create_dataset('qvel', (max_timesteps, 18))
        action = root.create_dataset('action', (max_timesteps, 18))

        for name, array in data_dict.items():
            root[name][...] = array
    print(f'Saving: {time.time() - t0:.1f} secs\n')


    cam_names = list(video[0].keys())
    h, w, _ = video[0][cam_names[0]].shape
    w = w * len(cam_names)
    fps = int(1/DT)
    out = cv2.VideoWriter(dataset_dir + '/hardcoded' + str(episode_idx) + '.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    for ts, image_dict in enumerate(video):
        images = []
        for cam_name in cam_names:
            image = image_dict[cam_name]
            image = image[:, :, [2, 1, 0]] # swap B and R channel
            images.append(image)
        images = np.concatenate(images, axis=1)
        out.write(images)
    out.release()


print(f'Saved to {dataset_dir}')
print(f'Success: {np.sum(success)} / {len(success)}')


episode_idx=0
Rollout out EE space scripted policy


PhysicsError: Physics state is invalid. Warning(s) raised: mjWARN_BADQACC